# Lab02 - Data manipulation, geostatistical analysis, and mapping

Learning goals

- develop skills in importing and modifying data using python.
- perform geographic statistical operations and transformations using python
- devlop skill in new and innovative means of mapping

---

This lab will look at an incredibly esoteric dataset and question. How has the geographic center of baseball teams shifted since 1900.

*Disclaimer* - I do not really even like baseball, but it is a cool dataset and also tells us a bit about how we have shifted geographically as a society.

*Disclaimer* - Liam really likes baseball, go Cubs



In [1]:
# This will install all of the required packages and modules
!apt install libspatialindex-dev --quiet
!apt install folium --quiet
!pip install rtree --quiet
!pip install geopandas --quiet
!pip install -U --no-deps mapclassify git+git://github.com/geopandas/geopandas.git@master --quiet
!pip install descartes --quiet # Helps geopandas plot polygons
!pip install rtree --quiet
!pip install pysal --quiet # Map classifiers for choropleth maps
!apt-get install software-properties-common python-software-properties > /dev/null
!add-apt-repository ppa:ubuntugis/ppa -y > /dev/null
!apt-get update > /dev/null
#!apt-get install -y --fix-missing python-gdal gdal-bin libgdal-dev > /dev/null
#!pip2 install OpticalRS > /dev/null

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet git://github.com/geopandas/geopandas.git /private/var/folders/1h/qds9mkm123b7fhmghg50vg6w62xq56/T/pip-req-build-6v7abxyz did not run successfully.
  │ exit code: 128
  ╰─> [3 lines of output]
      fatal: unable to connect to github.com:
      github.com[0: 140.82.114.4]: errno=Operation timed out
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet git://github.com/geopandas/geopandas.git /private/var/folders/1h/qds9mkm123b7fhmghg50vg6w62xq56/T/pip-req-build-6v7abxyz did not run successfull

In [2]:
# First lets import the packages we will need. Always put these up front!
%matplotlib inline
import numpy as np
import math
import folium #Folium is a library that allows us to create webmaps
# This will get all of the libraries (subcomponents of the packages and modules).  
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statistics
import scipy
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import variation
import geopandas as gpd
import mapclassify

/Users/f67f911/opt/anaconda3/envs/swe/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [ ]:
# Now upload the the Baseball_Decades.csv file. This is on D2L. Use this format for uploading small datasets in the future
from google.colab import files
uploaded = files.upload()

Saving Baseball_Decades.csv to Baseball_Decades.csv


In [ ]:
# read in the csv file and look at it
df = pd.read_csv('Baseball_Decades.csv')
print(df.head)
print(df.dtypes)

<bound method NDFrame.head of      Decade        Team   State  Latitude  Longitude
0      1900      Boston      MA  42.35866   -71.0567
1      1900    Brooklyn      NY  40.69245   -73.9904
2      1900     Chicago      IL  41.88425   -87.6324
3      1900  Cincinnati      OH  39.10713   -84.5041
4      1900    New York      NY  40.78200   -73.8317
..      ...         ...     ...       ...        ...
137    2010   St. Louis      MO  38.62775   -90.1996
138    2010   Tampa Bay      FL  27.58300   -82.6330
139    2010       Tempe      AZ  33.42551  -111.9370
140    2010     Toronto  Canada  43.65740   -79.4328
141    2010  Washington      DC  38.89037   -77.0320

[142 rows x 5 columns]>
Decade         int64
Team          object
State         object
Latitude     float64
Longitude    float64
dtype: object


It is important to look at the data types and understand the data, this will potentially mess you up later on. 

### Indexing

[Indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) on a pandas DataFrame allows you to subset or filter the data based upon a condition(s) that you provide.

A couple new concepts: To access all rows of one or more columns, we can simply specify the column names in brackets. Note that the printout doesn't print all of our values, but tells us the total length of the column and also tells us the data type of all elements of the column. Since each pandas DataFrame column is a NumPy array, all values in a column have the same data type.

In [ ]:
print(df['Team']) # just print out the column of the cities that had a team

0          Boston
1        Brooklyn
2         Chicago
3      Cincinnati
4        New York
          ...    
137     St. Louis
138     Tampa Bay
139         Tempe
140       Toronto
141    Washington
Name: Team, Length: 142, dtype: object


To access data by index labels, we must use the `.loc` attribute before the brackets. Recall from above that we currently have a `RangeIndex` with numbers that go from 0 to 141 with a step size of 1.

In [ ]:
df.loc[0] # just grab the first row

Decade           1900
Team           Boston
State              MA
Latitude     42.35866
Longitude    -71.0567
Name: 0, dtype: object

In [ ]:
df.loc[70:80] # grab rows 10 - 20

,Decade,Team,State,Latitude,Longitude
70,1980,Minneapolis,MN,44.97903,-93.2649
71,1980,Montreal,Canada,45.51720,-73.5499
72,1980,New York,NY,40.78200,-73.8317
73,1980,New York,NY,40.78200,-73.8317
74,1980,Oakland,CA,37.80508,-122.2730
75,1980,Philadelphia,PA,39.95228,-75.1625
76,1980,Pittsburgh,PA,40.43834,-79.9975
77,1980,San Diego,CA,32.71569,-117.1620
78,1980,San Francisco,CA,37.77712,-122.4200
79,1980,Seattle,WA,47.60357,-122.3290


In [ ]:
df.loc[10:20, ['Team','Decade']] # grab rows 10 - 20, but just the team and decade columns. 
# Notice how the column order was switched.

,Team,Decade
10,Brooklyn,1920
11,Chicago,1920
12,Chicago,1920
13,Cincinnati,1920
14,Cleveland,1920
15,Detroit,1920
16,New York,1920
17,New York,1920
18,Philadelphia,1920
19,Philadelphia,1920


We can use conditional statements and boolean logic in combination with `.loc` as well. Instead of the English words `and` and `or` like we used in Lab 1 conditional statements, use `&` and `|` characters for bitwise comparisons in the `.loc` statements. Also note that multiple logical statements each need to be enclosed in their own parentheses:

Lets find all of the data for 1960...

In [ ]:
teams_1960s = df.loc[df['Decade'] == 1960]
teams_1960s

,Decade,Team,State,Latitude,Longitude
0,1900,Boston,MA,42.35866,-71.0567
1,1900,Brooklyn,NY,40.69245,-73.9904
2,1900,Chicago,IL,41.88425,-87.6324
3,1900,Cincinnati,OH,39.10713,-84.5041
4,1900,New York,NY,40.78200,-73.8317
5,1900,Philadelphia,PA,39.95228,-75.1625
6,1900,Pittsburgh,PA,40.43834,-79.9975
7,1900,St. Louis,MO,38.62775,-90.1996


Let's just look at the teams in the 1960s that were in Ohio.

In [ ]:
# Combine a numerical value (Decade) with a text string (State)
data_1960_OH = df.loc[(df['Decade'] == 1960) & df.State.str.contains('OH')]
data_1960_OH

,Decade,Team,State,Latitude,Longitude
44,1960,Cincinnati,OH,39.10713,-84.5041
45,1960,Cleveland,OH,41.50475,-81.6907


In [ ]:
# Combine a numerical value (Decade) with a text string (State)
data_1900_NY = df.loc[(df['Decade'] == 1900) & df.State.str.contains('NY')]
data_1900_NY

,Decade,Team,State,Latitude,Longitude
1,1900,Brooklyn,NY,40.69245,-73.9904
4,1900,New York,NY,40.78200,-73.8317


In [ ]:
# Combine a numerical value (Decade) with a text string (State)
data_2000_NY = df.loc[(df['Decade'] == 2000) & df.State.str.contains('NY')]
data_2000_NY

,Decade,Team,State,Latitude,Longitude
99,2000,New York,NY,40.782,-73.8317
100,2000,New York,NY,40.782,-73.8317


One of the things to notice are the differences between the following two lines of code:

```
df.loc[df['Decade'] == 1960]
```

```
teams_1960s = df.loc[df['Decade'] == 1960]
```
While both code blocks might look the same, the first code block simply displays the selection, like a filter in Excel. The second code block actually creates a new variable from the subset.

## Questions
1. What team is saved in the 75th row? 
## Philadelphia is in the 74th row.
2. How many baseball teams were in NY state in 1900? How many teams were in NY state in 2000?
## There are two baseball teams in 1900 and also two in 2000.


## Getting the Latitude and Longitude into a cartesian surface. 

While it is easier for us to visualize lat & lon, the reality is that it is good for a flat map, not something like a sphere. realities is that it is not great for measuring distance on something like a sphere. Because of the we need to convert the Latitude and Longitude column into radians and then into a Cartesian plane. This will give our locations a numeric values rather than and coordinate value.

Convert Lat and Lon from degrees to radians using the general formulas.

```
lat_rad = lat * PI/180
lon_rad = lon * PI/180
```
Now we need to get into a Cartesian space.

```
X = cos(lat_rad)*cos(lon_rad)
Y = cos(lat_rad)*sin(lon_rad)
Z = sin(lat_rad)
```

So now let's turn that into executable code. To see how that might work, you need to run the following code in a new code cell:



```
demo = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
demo["shiny_new_column"] = df["a"] * df["b"]

print(demo)
```


Now let's roll with this on our Baseball dataset. 



In [ ]:
PI = math.pi # set the PI variable using the math package for simplicity 
PI

3.141592653589793

In [ ]:
# Finish this cell block for lon_rad
df["lat_rad"] = df["Latitude"] * PI/180

# You insert code for lon_rad here
df["lon_rad"] = df["Longitude"] * PI/180


Now convert the X,Y,Z cartesian space.

In [ ]:
df['X'] = np.cos(df.lat_rad)*np.cos(df.lon_rad)
df['Y'] = np.cos(df.lat_rad)*np.sin(df.lon_rad)
df['Z'] = np.sin(df.lat_rad)

# You complete the code for Y and Z


Alright so now we have our dataframe all set, now we just have to compute the geographic mean using the formulas:

$\overline{X} = \frac{X_{n}}{n}$


$\overline{Y} = \frac{Y_{n}}{n}$

$\overline{Z} = \frac{Z_{n}}{n}$

for each decade...

So  we need to do our analysis on the subset out each decade.

Hmm...

We need to go back to our indexing expertise. 

In [ ]:
print(df)

     Decade        Team   State  ...         X         Y         Z
0      1900      Boston      MA  ...  0.239884 -0.698921  0.673769
1      1900    Brooklyn      NY  ...  0.209116 -0.728813  0.651998
2      1900     Chicago      IL  ...  0.030756 -0.743860  0.667628
3      1900  Cincinnati      OH  ...  0.074318 -0.772401  0.630772
4      1900    New York      NY  ...  0.210850 -0.727251  0.653183
..      ...         ...     ...  ...       ...       ...       ...
137    2010   St. Louis      MO  ... -0.002722 -0.781213  0.624258
138    2010   Tampa Bay      FL  ...  0.113651 -0.879024  0.463033
139    2010       Tempe      AZ  ... -0.311797 -0.774173  0.550852
140    2010     Toronto  Canada  ...  0.132678 -0.711211  0.690345
141    2010  Washington      DC  ...  0.174667 -0.758497  0.627832

[142 rows x 10 columns]


In [ ]:
data_1960X = np.mean(df.loc[(df['Decade'] == 1960)])
print(data_1960X)

Decade       1960.000000
Latitude       40.064213
Longitude     -86.971987
lat_rad         0.699252
lon_rad        -1.517948
X               0.038300
Y              -0.740331
Z               0.643188
dtype: float64


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3370: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


While that was pretty sweet, doing this for all of the decades will be a pain. Plus then you are going to have a variable (like data_1960X for each of the decades. 

This is where the power of computing really shows its strengths.

Pandas `groupby` groups data according to the categories and apply a function to the categories.

Let me show you what I mean.

In [ ]:
geogMean = df.groupby(['Decade']).mean()
print(geogMean)

         Latitude  Longitude   lat_rad   lon_rad         X         Y         Z
Decade                                                                        
1900    40.480357 -79.546863  0.706516 -1.388355  0.136338 -0.742877  0.649048
1920    40.635912 -79.751781  0.709230 -1.391931  0.133634 -0.741790  0.651082
1940    40.635912 -79.751781  0.709230 -1.391931  0.133634 -0.741790  0.651082
1960    40.064213 -86.971987  0.699252 -1.517948  0.038300 -0.740331  0.643188
1980    39.338697 -91.991654  0.686590 -1.605557 -0.028153 -0.737364  0.631988
2000    38.485363 -92.402903  0.671696 -1.612735 -0.032835 -0.747164  0.619978
2010    38.179712 -92.490617  0.666362 -1.614266 -0.033013 -0.750535  0.615800


Holy Cow - that was slick. But it looks like `Decade` is not a column name, and is instead an index. (This caused me a lot of suffering while developing this lab!)

> Let's redo our work, but clean up our dataset.

In [ ]:
geogMean = df.groupby(['Decade'],as_index=False).mean() # Don't make decade an Index! Instead it should be a column name.
print(geogMean)


   Decade   Latitude  Longitude  ...         X         Y         Z
0    1900  40.480357 -79.546863  ...  0.136338 -0.742877  0.649048
1    1920  40.635912 -79.751781  ...  0.133634 -0.741790  0.651082
2    1940  40.635912 -79.751781  ...  0.133634 -0.741790  0.651082
3    1960  40.064213 -86.971987  ...  0.038300 -0.740331  0.643188
4    1980  39.338697 -91.991654  ... -0.028153 -0.737364  0.631988
5    2000  38.485363 -92.402903  ... -0.032835 -0.747164  0.619978
6    2010  38.179712 -92.490617  ... -0.033013 -0.750535  0.615800

[7 rows x 8 columns]


In [ ]:
df.head

<bound method NDFrame.head of      Decade        Team   State  ...         X         Y         Z
0      1900      Boston      MA  ...  0.239884 -0.698921  0.673769
1      1900    Brooklyn      NY  ...  0.209116 -0.728813  0.651998
2      1900     Chicago      IL  ...  0.030756 -0.743860  0.667628
3      1900  Cincinnati      OH  ...  0.074318 -0.772401  0.630772
4      1900    New York      NY  ...  0.210850 -0.727251  0.653183
..      ...         ...     ...  ...       ...       ...       ...
137    2010   St. Louis      MO  ... -0.002722 -0.781213  0.624258
138    2010   Tampa Bay      FL  ...  0.113651 -0.879024  0.463033
139    2010       Tempe      AZ  ... -0.311797 -0.774173  0.550852
140    2010     Toronto  Canada  ...  0.132678 -0.711211  0.690345
141    2010  Washington      DC  ...  0.174667 -0.758497  0.627832

[142 rows x 10 columns]>

##Questions
3. What is the mean x,y, and z coordinate for 2010?
 The XY coordinate for 2010 is: 38.179712 -92.490617


# Converting back to lat and long

As we discussed in class lat and lon are great for mapping things on a flat map, but do not do so well with measuring distances on a sphere. 

To display our mean centers on a webmap we have to take our x,y, and z measuremtns and return them to Lat and Long

Convert lat and lon to degrees from radians.

```
meanLat = meanLat * 180/PI
meanLon = meanLon * 180/PI
```

Convert average x, y, z coordinate to latitude and longitude. 

```
meanLon = arctan2(y, x)
Hyp = sqrt(x * x + y * y)
meanLat = arctan2(z, hyp)
```





In [ ]:
## df["lat_rad"] = df["Latitude"] * PI/180

In [ ]:
geogMean["meanLon"] = (np.arctan2(geogMean.Y, geogMean.X)) * (180/PI)
Hyp = np.sqrt(geogMean.X * geogMean.X + geogMean.Y * geogMean.Y)
geogMean["meanLat"] = (np.arctan2(geogMean.Z, Hyp)) * (180/PI)
print(geogMean)

   Decade   Latitude  Longitude  ...         Z    meanLon    meanLat
0    1900  40.480357 -79.546863  ...  0.649048 -79.600404  40.673906
1    1920  40.635912 -79.751781  ...  0.651082 -79.787631  40.820851
2    1940  40.635912 -79.751781  ...  0.651082 -79.787631  40.820851
3    1960  40.064213 -86.971987  ...  0.643188 -87.038508  40.945700
4    1980  39.338697 -91.991654  ...  0.631988 -92.186529  40.578961
5    2000  38.485363 -92.402903  ...  0.619978 -92.516315  39.657896
6    2010  38.179712 -92.490617  ...  0.615800 -92.518601  39.341093

[7 rows x 10 columns]


In [ ]:
#Create a column for the mean latitude and mean longitude in our geogMean dataframe here
geogClean = geogMean[['Decade', 'meanLon', 'meanLat']]
print(geogClean)

#HINT: you may need to make an additioanl column to calculate mean lattitude

   Decade    meanLon    meanLat
0    1900 -79.600404  40.673906
1    1920 -79.787631  40.820851
2    1940 -79.787631  40.820851
3    1960 -87.038508  40.945700
4    1980 -92.186529  40.578961
5    2000 -92.516315  39.657896
6    2010 -92.518601  39.341093


Now lets clean up our dataset. We only care about the Mean Latitude, Mean Longitude, and Decade columns. Drop the other columns from the geogMean dataframe

In [ ]:
geogClean['Decade'] = geogClean['Decade'].apply(str) #Confirms that the decade variable a string

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Now let's map out these data using [Folium](/https://python-visualization.github.io/folium/quickstart.html#Vincent/Vega-and-Altair/VegaLite-Markers).

In [ ]:
#Create a webmap here that displays the mean centers of MLB teams over time. 
#create a map

m = folium.Map(prefer_canvas=True, location=[40.57861,-85.186529],
zoom_start=6, tiles="Stamen Toner")

for i in range (0,len(geogClean)):
  folium.Marker(
      location = [geogClean.iloc[i]['meanLat'],geogClean.iloc[i]['meanLon']], 
      popup=geogClean.iloc[i]['Decade']
                
  ).add_to(m)

m


##Questions
4. Our map shows an unweighted spatial **Mean** of professional baseball teams over time. What do you think might change if we graphed the spatial **Median** of the teams? Why?

I think if we mapped the Median of the teams over time, it would likely shift towards the highest concentrations of teams per area, which is likely on the East Coast, trending in the same direction as the population map we looked at on Monday. This is because calculating the spatial Median minimizes the distance to every other point and will be less affected by outliers.

5. Liam is a huge baseball fan. According to our map, where should he move to be as close to as many professional teams as possible? 

Liam should move to Moberly Missouri. Maybe, Kansas City or Springfield. Because if the trends we have seen over the last hundred years continue to be true we can expect the mean center to move south, and to the west.

6. Liam wants to minimize his travel time to see as many games as possible. Do you think he should decide where to move using the spatial mean of the teams or the spatial median? Why?

I think he should decide where to move based on the spatial median. This is because it takes into account the shortest euclidian distance between all other teams. Where the median is just the central point of the sum of all teams.